In [1]:
import pandas as pd


In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
csv_file1 = "../Project/2019_nCov_data.csv"
corona_data_df=pd.read_csv(csv_file1)
corona_data_df.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,1/22/2020 12:00,Anhui,China,1/22/2020 12:00,1,0,0
1,2,1/22/2020 12:00,Beijing,China,1/22/2020 12:00,14,0,0
2,3,1/22/2020 12:00,Chongqing,China,1/22/2020 12:00,6,0,0
3,4,1/22/2020 12:00,Fujian,China,1/22/2020 12:00,1,0,0
4,5,1/22/2020 12:00,Gansu,China,1/22/2020 12:00,0,0,0


In [4]:
csv_file2 = "../Project/time_series_2019_ncov_confirmed - 1.csv"
confirmed_data_df=pd.read_csv(csv_file2)
confirmed_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 22:04,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,1.0,9.0,15.0,15.0,39.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,36.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,30.05718,107.8740,5.0,6.0,9.0,27.0,27.0,57.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
3,Fujian,Mainland China,26.07783,117.9895,NaN,1.0,5.0,5.0,10.0,10.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,2.0,2.0,2.0,4.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83


In [5]:
confirmed_data_df.drop(['Lat','Long'], axis=1)

,Province/State,Country/Region,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,1/25/2020 12:00,1/25/2020 22:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 22:04,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
0,Anhui,Mainland China,NaN,1.0,9.0,15.0,15.0,39.0,39.0,60.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
1,Beijing,Mainland China,10.0,14.0,22.0,26.0,36.0,36.0,41.0,51.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,5.0,6.0,9.0,27.0,27.0,57.0,57.0,75.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
3,Fujian,Mainland China,NaN,1.0,5.0,5.0,10.0,10.0,18.0,18.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
4,Gansu,Mainland China,NaN,NaN,2.0,2.0,2.0,4.0,4.0,7.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
68,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
69,NaN,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
70,"Madison, WI",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,1,1,1,1,1,1


In [6]:
csv_file3 = "../Project/time_series_2019_ncov_deaths.csv"
deaths_data_df=pd.read_csv(csv_file3)
deaths_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 10:24,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
1,Beijing,Mainland China,40.18238,116.4142,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
2,Chongqing,Mainland China,30.05718,107.8740,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
3,Fujian,Mainland China,26.07783,117.9895,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2


In [7]:
deaths_data_df.drop(['Lat','Long'], axis=1)

,Province/State,Country/Region,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,1/25/2020 12:00,1/25/2020 22:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 10:24,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
0,Anhui,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
1,Beijing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
2,Chongqing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
3,Fujian,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Gansu,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
68,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
69,NaN,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
70,"Madison, WI",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [8]:
csv_file4 = "../Project/time_series_2019_ncov_recovered.csv"
recovered_data_df=pd.read_csv(csv_file4)
recovered_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 10:24,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,23.0,34.0,34.0,47.0,47.0,59,59,72,73
1,Beijing,Mainland China,40.18238,116.4142,NaN,NaN,NaN,NaN,1.0,1.0,...,24.0,31.0,31.0,31.0,33.0,34.0,34,37,37,44
2,Chongqing,Mainland China,30.05718,107.8740,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,15.0,24.0,24.0,31.0,31.0,39,39,50,51
3,Fujian,Mainland China,26.07783,117.9895,NaN,NaN,NaN,NaN,NaN,NaN,...,11.0,11.0,14.0,14.0,20.0,20.0,24,26,35,35
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,6.0,6.0,6.0,9.0,9.0,12,12,15,16


In [9]:
recovered_data_df.drop(['Lat','Long'], axis=1)

,Province/State,Country/Region,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,1/25/2020 12:00,1/25/2020 22:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 10:24,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
0,Anhui,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,23.0,34.0,34.0,47.0,47.0,59,59,72,73
1,Beijing,Mainland China,NaN,NaN,NaN,NaN,1.0,1.0,2.0,2.0,...,24.0,31.0,31.0,31.0,33.0,34.0,34,37,37,44
2,Chongqing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,15.0,24.0,24.0,31.0,31.0,39,39,50,51
3,Fujian,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.0,11.0,14.0,14.0,20.0,20.0,24,26,35,35
4,Gansu,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,6.0,6.0,6.0,9.0,9.0,12,12,15,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
68,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
69,NaN,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
70,"Madison, WI",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [10]:
countries =  corona_data_df['Country'].unique()
countries

array(['China', 'US', 'Japan', 'Thailand', 'South Korea',
       'Mainland China', 'Hong Kong', 'Macau', 'Taiwan', 'Singapore',
       'Philippines', 'Malaysia', 'Vietnam', 'Australia', 'Mexico',
       'Brazil', 'France', 'Nepal', 'Canada', 'Cambodia', 'Sri Lanka',
       'Ivory Coast', 'Germany', 'Finland', 'United Arab Emirates',
       'India', 'Italy', 'Sweden', 'Russia', 'Spain', 'UK', 'Belgium',
       'Others'], dtype=object)

In [11]:
Province_State = corona_data_df['Province/State'].unique()
Province_State

array(['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong',
       'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan',
       'Hong Kong', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu',
       'Jiangxi', 'Jilin', 'Liaoning', 'Macau', 'Ningxia', 'Qinghai',
       'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Taiwan',
       'Tianjin', 'Tibet', 'Washington', 'Xinjiang', 'Yunnan', 'Zhejiang',
       nan, 'Chicago', 'Illinois', 'California', 'Arizona', 'Ontario',
       'New South Wales', 'Victoria', 'Bavaria', 'British Columbia',
       'Queensland', 'Chicago, IL', 'South Australia', 'Boston, MA',
       'Los Angeles, CA', 'Orange, CA', 'Santa Clara, CA', 'Seattle, WA',
       'Tempe, AZ', 'Toronto, ON', 'San Benito, CA', 'London, ON',
       'Madison, WI', 'Cruise Ship', 'Diamond Princess cruise ship'],
      dtype=object)

In [12]:
country_df= corona_data_df.groupby([ 'Country'])
country_df

In [13]:
total_cdr_country_df = country_df["Confirmed","Deaths","Recovered"].sum()
total_cdr_country_df

,Confirmed,Deaths,Recovered
Country,,,
Australia,179,0,24
Belgium,7,0,0
Brazil,0,0,0
Cambodia,15,0,0
Canada,66,0,0
China,549,0,0
Finland,13,0,0
France,106,0,0
Germany,140,0,0


In [14]:
new_corona_data_df = corona_data_df[['Sno','Province/State','Country','Confirmed','Deaths','Recovered']].copy()
new_corona_data_df

,Sno,Province/State,Country,Confirmed,Deaths,Recovered
0,1,Anhui,China,1,0,0
1,2,Beijing,China,14,0,0
2,3,Chongqing,China,6,0,0
3,4,Fujian,China,1,0,0
4,5,Gansu,China,0,0,0
...,...,...,...,...,...,...
1194,1195,"Los Angeles, CA",US,1,0,0
1195,1196,"Madison, WI",US,1,0,0
1196,1197,"Orange, CA",US,1,0,0
1197,1198,"Seattle, WA",US,1,0,1


In [15]:
new_corona_data_df.drop(['Sno'], axis=1)

,Province/State,Country,Confirmed,Deaths,Recovered
0,Anhui,China,1,0,0
1,Beijing,China,14,0,0
2,Chongqing,China,6,0,0
3,Fujian,China,1,0,0
4,Gansu,China,0,0,0
...,...,...,...,...,...
1194,"Los Angeles, CA",US,1,0,0
1195,"Madison, WI",US,1,0,0
1196,"Orange, CA",US,1,0,0
1197,"Seattle, WA",US,1,0,1


In [16]:
confirmed_data_df = pd.read_csv(csv_file2)
confirmed_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 22:04,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,1.0,9.0,15.0,15.0,39.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,36.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,30.05718,107.8740,5.0,6.0,9.0,27.0,27.0,57.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
3,Fujian,Mainland China,26.07783,117.9895,NaN,1.0,5.0,5.0,10.0,10.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,2.0,2.0,2.0,4.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83


In [17]:
new_confirmed_data_df = confirmed_data_df.dropna()
new_confirmed_data_df

,Province/State,Country/Region,Lat,Long,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 22:04,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,36.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,30.05718,107.8740,5.0,6.0,9.0,27.0,27.0,57.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
5,Guangdong,Mainland China,23.33841,113.4220,17.0,26.0,32.0,53.0,53.0,78.0,...,895.0,944.0,970.0,970.0,1034,1075,1095,1120,1131,1151
12,Hubei,Mainland China,30.97564,112.2707,270.0,444.0,444.0,549.0,549.0,729.0,...,16678.0,19665.0,19665.0,19665.0,22112,24953,24953,27100,27100,29631
13,Hunan,Mainland China,27.61041,111.7088,1.0,4.0,9.0,24.0,24.0,43.0,...,661.0,711.0,711.0,711.0,772,803,803,838,838,879
16,Jiangxi,Mainland China,27.61401,115.7221,2.0,2.0,7.0,7.0,18.0,18.0,...,548.0,600.0,600.0,600.0,661,698,698,740,740,771
22,Shandong,Mainland China,36.34377,118.1529,1.0,2.0,6.0,9.0,15.0,21.0,...,307.0,343.0,347.0,347.0,386,407,416,435,444,459
23,Shanghai,Mainland China,31.20327,121.4554,9.0,9.0,16.0,20.0,20.0,33.0,...,243.0,254.0,257.0,257.0,277,281,286,292,293,295
25,Sichuan,Mainland China,30.61714,102.7103,2.0,5.0,8.0,15.0,15.0,28.0,...,301.0,321.0,321.0,321.0,344,363,364,386,386,405
26,Tianjin,Mainland China,39.29362,117.3330,2.0,4.0,4.0,5.0,8.0,8.0,...,69.0,69.0,78.0,78.0,81,88,88,90,90,94


In [18]:
new_confirmed_data_df.drop(['Lat','Long'], axis=1)

,Province/State,Country/Region,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,1/25/2020 12:00,1/25/2020 22:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 22:04,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
1,Beijing,Mainland China,10.0,14.0,22.0,26.0,36.0,36.0,41.0,51.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,5.0,6.0,9.0,27.0,27.0,57.0,57.0,75.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
5,Guangdong,Mainland China,17.0,26.0,32.0,53.0,53.0,78.0,78.0,98.0,...,895.0,944.0,970.0,970.0,1034,1075,1095,1120,1131,1151
12,Hubei,Mainland China,270.0,444.0,444.0,549.0,549.0,729.0,761.0,1052.0,...,16678.0,19665.0,19665.0,19665.0,22112,24953,24953,27100,27100,29631
13,Hunan,Mainland China,1.0,4.0,9.0,24.0,24.0,43.0,43.0,69.0,...,661.0,711.0,711.0,711.0,772,803,803,838,838,879
16,Jiangxi,Mainland China,2.0,2.0,7.0,7.0,18.0,18.0,18.0,36.0,...,548.0,600.0,600.0,600.0,661,698,698,740,740,771
22,Shandong,Mainland China,1.0,2.0,6.0,9.0,15.0,21.0,27.0,39.0,...,307.0,343.0,347.0,347.0,386,407,416,435,444,459
23,Shanghai,Mainland China,9.0,9.0,16.0,20.0,20.0,33.0,33.0,40.0,...,243.0,254.0,257.0,257.0,277,281,286,292,293,295
25,Sichuan,Mainland China,2.0,5.0,8.0,15.0,15.0,28.0,28.0,44.0,...,301.0,321.0,321.0,321.0,344,363,364,386,386,405
26,Tianjin,Mainland China,2.0,4.0,4.0,5.0,8.0,8.0,10.0,13.0,...,69.0,69.0,78.0,78.0,81,88,88,90,90,94


In [37]:
rds_connection_string = "postgres:Glacy1234@localhost:5432/Corona_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [38]:
total_cdr_country_df.to_sql(name='total_cdr_country', con=engine, if_exists='append', index=False)

In [39]:
new_corona_data_df.to_sql(name='corona_data', con=engine, if_exists='append', index=False)

In [40]:
confirmed_data_df.to_sql(name='confirmed_data', con=engine, if_exists='append', index=False)

In [41]:
deaths_data_df .to_sql(name='deaths_data', con=engine, if_exists='append', index=False)

In [42]:
recovered_data_df .to_sql(name='recovered_data', con=engine, if_exists='append', index=False)

In [43]:
pd.read_sql_query('select * from total_cdr_country', con=engine)

,Confirmed,Deaths,Recovered
0,179,0,24
1,7,0,0
2,0,0,0
3,15,0,0
4,66,0,0
5,549,0,0
6,13,0,0
7,106,0,0
8,140,0,0
9,300,7,0


In [44]:
pd.read_sql_query('select * from corona_data', con=engine)

,Sno,Province/State,Country,Confirmed,Deaths,Recovered
0,1,Anhui,China,1,0,0
1,2,Beijing,China,14,0,0
2,3,Chongqing,China,6,0,0
3,4,Fujian,China,1,0,0
4,5,Gansu,China,0,0,0
...,...,...,...,...,...,...
1194,1195,"Los Angeles, CA",US,1,0,0
1195,1196,"Madison, WI",US,1,0,0
1196,1197,"Orange, CA",US,1,0,0
1197,1198,"Seattle, WA",US,1,0,1


In [45]:
pd.read_sql_query('select * from confirmed_data', con=engine)

,Province/State,Country/Region,Lat,Long,1/21/2020 22:00,1/22/2020 12:00,1/23/2020 12:00,1/24/2020 0:00,1/24/2020 12:00,1/25/2020 0:00,...,2/5/2020 9:00,2/5/2020 23:00,2/6/2020 9:00,2/6/2020 14:20,2/7/2020 20:13,2/7/2020 22:50,2/8/2020 22:04,2/8/2020 23:04,2/9/2020 10:30,2/9/2020 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,1.0,9.0,15.0,15.0,39.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,36.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,30.05718,107.8740,5.0,6.0,9.0,27.0,27.0,57.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
3,Fujian,Mainland China,26.07783,117.9895,NaN,1.0,5.0,5.0,10.0,10.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,2.0,2.0,2.0,4.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,"Boston, MA",US,42.36010,-71.0589,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
68,"San Benito, CA",US,36.57610,-120.9876,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
69,None,Belgium,50.50390,4.4699,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
70,"Madison, WI",US,43.07310,-89.4012,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,1,1,1,1,1,1


In [46]:
engine.table_names()

['corona_china_cdr_',
 'total_cdr_country',
 'corona_data',
 'confirmed_data',
 'deaths_data',
 'recovered_data']